<a href="https://colab.research.google.com/github/AlexeyK12/Data_scientist-Sberuniversity/blob/main/RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендательные системы

Источник: https://colab.research.google.com/drive/1-3XVdCprzlzeWtd-T6Zr3WBdnd1bDQ91?usp=sharing#scrollTo=EpUyM57bxyqO

Идея рекомендательных систем на основе ближайшего соседа – использовать предпочтения некоторой группы людей, похожих на данного пользователя, чтобы рекомендовать ему что-либо.

У такой модели много применений:
    
   * рекомендация товаров интернет-магазинов
   * рекомендация музыки или фильмов
   * рекомендация людей/контента в социальных сетях

![](amazon.jpg)

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1buQlTY5D-uT18rsjmp-hO60sjtN0rpXg"
alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>


**Пример рекомендательной системы Amazon.com**

![](imdb.png)

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1zEIj_g4A-5assEIxY7UCQpLMsaETmyCi"
alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>


**Рекомендация фильмов от IMDb**

Естественный способ получить рекомендацию о чем-либо - спросить мнение об этом у друзей или любых людей, которым нравится то же, что и вам. Эту идею можно использовать и для машины: для каждого человека алгоритм просматривает большую группу людей и ищет в ней подгруппу с похожим на данного человека вкусом. Далее создается список того, что еще нравится этим людям, а затем человеку рекомендуются предложения из этого списка. Такой алгоритм называется алгоритмом **коллаборативной фильтрации**.

**Отключаем предупреждения**

Библиотека **warnings** отвечает за то, какие предупреждения (warnings) о работе будут выводиться пользователю.
FutureWarning - предупреждения о том, как изменится работа библиотек в будущих версиях. Такие предупреждения мы будем игнорировать.
Чтобы включить режим игнорирования, мы отбираем все предупреждения из категории FutureWarning и выбираем для них действия 'ignore'.
Это делается вызовом функции simplefilter c задание двух атрибутов: действия action и категории предупреждений category.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Для красивого вывода на экран сложных объектов будем использовать **pprint**

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

### Подготовка данных

Чтобы хранить сразу много предпочтений для каждого человека, удобнее всего воспользоваться **вложенным словарем**.

Самый простой способ создать словарь в python - использовать фигурные скобки {}. Данные в словаре хранятся в формате ключ – значение, разделенные двоеточием:

```python
dict = {'key': 'value'}
```

Мы будем работать с *вложенным словарем* кинокритиков и выставленных ими оценок для небольшого набора данных о фильмах. Построим его следующим образом:

1) для каждого кинокритика создаем словарь оценок фильмов в формате
   ```python
    scores_dict = {
        'film_1': 'score_1',
        'film_2': 'score_2',
        ...
    }
   ```
   
2) создаем словарь кинокритиков, где в качестве значений будет соответствующий ему словарь оценок
   ```python
     critics = {
         'name_1': 'scores_dict_1',
         'name_2': 'scores_dict_2',
         ...
     }
   ```

In [ ]:
# Словарь кинокритиков и выставленных ими оценок
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0,
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [ ]:
pp.pprint(critics)

{   'Claudia Puig': {   'Just My Luck': 3.0,
                        'Snakes on a Plane': 3.5,
                        'Superman Returns': 4.0,
                        'The Night Listener': 4.5,
                        'You, Me and Dupree': 2.5},
    'Gene Seymour': {   'Just My Luck': 1.5,
                        'Lady in the Water': 3.0,
                        'Snakes on a Plane': 3.5,
                        'Superman Returns': 5.0,
                        'The Night Listener': 3.0,
                        'You, Me and Dupree': 3.5},
    'Jack Matthews': {   'Lady in the Water': 3.0,
                         'Snakes on a Plane': 4.0,
                         'Superman Returns': 5.0,
                         'The Night Listener': 3.0,
                         'You, Me and Dupree': 3.5},
    'Lisa Rose': {   'Just My Luck': 3.0,
                     'Lady in the Water': 2.5,
                     'Snakes on a Plane': 3.5,
                     'Superman Returns': 3.5,
                 

### Критерий похожести

Как мы уже говорили, чтобы выделить подгруппу людей с похожим вкусом, необходимо как-то определить, насколько люди похожи. В нашем случае мы будем сравнивать оценки людей у одинаковых фильмов. Мы рассмотрим два способа сравнения. С помощью
- расстояния Евклида
- корреляции Пирсона

**Оценка по евклидову расстоянию**

Евклидово расстояние для точек $x = (x_1, x_2, ... , x_n)$ и $y = (y_1, y_2, ... , y_n)$ определяется следующим образом:

\begin{equation}
d(x, y) = \sqrt{\sum_{i = 1}^n (x_i - y_i)^2}
\end{equation}

Для понимания расстояния Евклида рассмотрим случай, когда *n = 2*. То есть каждый объект выборки (точка) описывается двумя параметрами (координатами). $x = (x_1, x_2)$ и $y = (y_1, y_2)$. Тогда Евклидово расстояние равно расстоянию между точками на плоскости.

\begin{equation}
d(x, y) = \sqrt{\sum_{i = 1}^n (x_i - y_i)^2} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
\end{equation}

Или, если представить задачу геометрически:

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1BXgfArl4T8N2M4eFur3JrsGloGWDDAlb"
alt="IMAGE ALT TEXT HERE" width="300" border="0" /></a>


Расстояние, вычисленное по этой формуле, будет тем меньше, чем больше сходство людей (чем ближе точки). Нам же нужна функция, значение которой будет, наоборот, большое, если люди сильно похожи друг на друга. То есть схожесть больше (от 0 до 1), если точки ближе друг другу


Для этого будем использовать функцию "похожести" в таком виде:

\begin{equation}
\text{similarity }(x, y) = \dfrac{1}{1 + d(x, y)},
\end{equation}

где $d(x, y)$ - расстояние Евклида.

Далее реализуем несколько простых функций для коллаборативной фильтрации на основе ближайшего соседа

Функция для подсчета сходства двух критиков на основе расстояния Евклида **sim_distance()**. Функция принимает на вход три аргумента:
* словарь критиков
* имя первого критика для сравнения
* имя второго критика для сравнения

In [ ]:
from scipy.stats import pearsonr
from math import sqrt
from math import isnan

def similar_films(critics_dict, person1, person2):
    # создаем пустой список фильмов
    sim_film = []

    # для всех фильмов у person1...
    for film in critics_dict[person1]:
        # если такой фильм оценивался person 2...
        if film in critics_dict[person2]:
            # добавляем этот фильм в список sim_film
            sim_film.append(film)

    return sim_film

def sim_distance(critics_dict, person1, person2):
    # используем написанную функцию для получения фильмов, оцененных обоими критиками
    sim_films = similar_films(critics_dict, person1, person2)

    # если нет ни одной общей оценки, то есть длина sim_film равна 0 - вернуть 0
    if len(sim_films) == 0:
        return 0

    # для каждого фильма из sim_film посчитаем евклидово расстояние и просуммируем все полученные значения
    sum_of_euclead_dist = 0

    for film in sim_films:
        sum_of_euclead_dist = sum_of_euclead_dist + pow(critics_dict[person1][film] - critics_dict[person2][film], 2)

    mean_of_euclead_dist = sum_of_euclead_dist / len(sim_films)

    # считаем похожесть
    score = 1 / (1 + mean_of_euclead_dist)

    return score

Посмотрим, как работает эта функция для двух критиков Lisa Rose и Toby:

In [ ]:
sim_distance(critics,'Lisa Rose','Toby')

0.46153846153846145

Эта функция работает также как:

\begin{equation}
\text{similarity }(x, y) = \dfrac{1}{1 + d(x, y)},
\end{equation}

где $d(x, y)$ - расстояние Евклида.

Однако, по осям будет фильмы и оценки от критиков. Если у критиков нет одинаковых фильмов, то мы этих критиков не рассматриваем

**Коэффициент корреляции Пирсона**

Другой способ определять степень схожести - использовать коэффициент корреляции Пирсона. Этот коэффициент показывает, насколько два набора данных хорошо ложатся на прямую.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1qXZxG2sE3Y0oAcgayaEdN6nCYVunj_83"
alt="IMAGE ALT TEXT HERE" width="800" border="0" /></a>

Два кинокритика со средним значением корреляции 0.41 (слева) и два кинокритика с высоким значением корреляции 0.75 (справа).  

Корреляция Пирсона считается по формуле:     
\begin{equation}
    \text{corr}(x, y) = \dfrac{\sum_{i = 1}^n (x_i - \overline x) (y_i - \overline y)}{\sqrt{\sum_{i = 1}^n (x_i - \overline x)^2 \sum_{i = 1}^n (y_i - \overline y)^2}}
\end{equation}
где $\overline x = \frac{1}{n}\sum_{j = 1}^n x_j, \overline y = \frac{1}{n}\sum_{j = 1}^n y_j$

По аналогии с функцией **sim_distance()** в модуле **recommendations** реализуем функцию для подсчета сходства двух критиков на основе корреляции Пирсона. Она называется **sim_pearson()**. Функция принимает на вход те же самые аргументы, что и **sim_distance()**:
* словарь критиков
* имя первого критика для сравнения
* имя второго критика для сравнения

In [ ]:
def sim_pearson(critics_dict, person1, person2):
    # получить список фильмов, оцененных обоими критиками
    sim_films = similar_films(critics_dict, person1, person2)

    # если нет ни одной общей оценки, вернуть 0
    if len(sim_films) < 2:
        return 0

    # получим оценки критиков для фильмов из пересечения
    scores1 = []
    scores2 = []
    for film in sim_films:
        scores1.append(critics_dict[person1][film])
        scores2.append(critics_dict[person2][film])

    # посчитаем коэффициент корреляции
    # берем [0] от результата, так как функция pearsonr() возвращает еще значение p-value
    r = pearsonr(scores1, scores2)[0]

    # если вдруг было деление на ноль и функция вернула nan, то присваиваем ноль
    if isnan(r):
        r = 0
    return r

Посмотрим, как работает эта функция для двух критиков Lisa Rose и Gene Seymour:

In [ ]:
sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

0.3960590171906697

### Ранжирование критиков

После того как мы выбрали критерий для сравнения двух людей, можно начать искать для каждого человека подгруппу других людей с наиболее похожими вкусами.

Для этого реализуем функцию **topMatches()**. Функция возвращает список n наилучших соответствий для человека из словаря critics.

Соответственно, аргументы у функции:
* словарь критиков *critics\_dict*
* имя персоны *person*, для которой подбираем соответствия
* число *n* наилучших соответствий

Кроме этого, так как мы хотим использовать различные расстояния, то в качестве аргумента мы будем также передавать имя функции-расстояния (*similarity*).  

In [ ]:
def top_matches(critics_dict, person, n=5, similarity=sim_pearson):
    # инициализируем лист оценок похожести
    scores = []

    # для каждого человека в словаре критиков
    for other in critics_dict:

        # если этот человек не тот, для которого мы подбираем соответствия
        if other != person:
            # добавить в scores похожесть, вычисленную с помощью функции, записанной в similarity, и имя человека
            scores.append((similarity(critics_dict, person, other), other))

    # отсортировать список по убыванию оценок
    scores.sort(reverse=True)  # аргумент reverse=True отвечает за убывающий порядок

    # возвращаем первые n человек
    return scores[0:n]

Пример вычисления топ-3 критиков похожих на Toby по корреляции Пирсона:


In [ ]:
top_matches(critics,'Toby',similarity=sim_pearson)

[(0.9912407071619302, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415642, 'Claudia Puig'),
 (0.6628489803598702, 'Jack Matthews'),
 (0.3812464258315117, 'Gene Seymour')]

Как работает эта функция:
*  считаем схожесть интересующего нас критика с каждым другим критиком используя функцию **similarity**
*  сортируем результаты
*  возвращаем топ **n** результатов

**Упражнение** Вычислите топ-3 критиков похожих на Toby по евклидову расстоянию:

In [ ]:
top_matches(critics,'Toby',similarity=sim_distance)

[(0.5714285714285714, 'Mick LaSalle'),
 (0.4800000000000001, 'Claudia Puig'),
 (0.46153846153846145, 'Lisa Rose'),
 (0.4444444444444444, 'Michael Phillips'),
 (0.2857142857142857, 'Jack Matthews')]

Выбор расстояния для подбора похожести зависит от конкретной задачи. Имеет смысл попробовать разные критерии и посмотреть, какой из них дает наилучший результат.

## Рекомендация фильмов (User-based подход)

Мы умеем находить людей с похожим мнением как у рассматриваемого человека. Но как получить конкретную рекомендацию с набором фильмов? Можно было бы просто посмотреть на то, какие фильмы понравились похожему человеку, и выбрать из них не просмотренные. Однако, такой способ не стабилен: можем, например, отобрать критика, которому по каким-то причинам понравился фильм, получивший негативные оценки от остальных.

Чтобы разрешить эту проблему, можно ранжировать сами фильмы, вычисляя взвешенную сумму оценок критиков. Мы будем брать оценку фильма у критика и умножать его на коэффициент похожести с тем, для которого мы строим рекомендацию. Затем полученный результат суммировать и нормировать.  

Идея такого подхода в том, что люди похожие на тебя вносят больший вклад в "рейтинг" фильма.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1x0QC1Blvr_ZugJ5BKPJdi2Bp30FvBxvL"
alt="IMAGE ALT TEXT HERE" width="500" border="0" /></a>

Этот подход реализован в функции **get_recommendations()**. Функция принимает на вход следующие аргументы:
* словарь критиков *critics\_dict*
* имя персоны *person*, для которой подбираем соответствия
* имя функции-критерия для сравнения


In [ ]:
def get_recommendations(prefs, person, similarity=sim_pearson):

    # инициализируем словари
    totals = {}
    simSums = {}

    # для каждого пользователя
    for other in prefs:

    # сравнивать person с person не нужно
        if other == person:
            continue
        # считаем похожесть
        sim = similarity(prefs, person, other)

        # игнорировать нулевые и отрицательные значения похожести
        if sim <= 0:
            continue

        # для каждого фильма в оцененных other
        for item in prefs[other]:

            # оценивать только фильмы, которые я еще не смотрел
            if item not in prefs[person] or prefs[person][item] == 0:

                # коэффициент подобия * Оценка
                totals.setdefault(item,0)
                totals[item] = totals[item] + prefs[other][item]*sim

                # cумма коэффициентов подобия
                simSums.setdefault(item,0)
                simSums[item] = simSums[item] + sim
    # создать нормированный список
    rankings = [(total/simSums[item], item) for item, total in totals.items( )]

    # вернуть отсортированный список
    rankings.sort(reverse=True)
    return rankings

# обращает матрицу предпочтений, чтобы строки соответствовали образцам
def transformPrefs(critics_dict):

    # инициилизируем новый словарь с фильмами и их оценками
    result = {}

    # для каждого критика в словаре
    for person in critics_dict:

        # для каждого оцененного им фильма
        for item in critics_dict[person]:

            # добавляем такой объект в словарь
            result.setdefault(item, {})
            # меняеи местами человека и предмет
            result[item][person] = critics_dict[person][item]
    return result

In [ ]:
get_recommendations(critics, 'Toby', similarity=sim_pearson)

[(3.347789526713101, 'The Night Listener'),
 (2.832549918264162, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

Как работает эта функция:
*  считаем схожесть пользователя с каждым другим пользователем
*  перебираем фильмы, которые смотрели другие пользователи, но не смотрел текущий пользователь
*  нормируем рейтинг фильмов на основе того насколько он понравился другим пользователям, и того насколько каждый из других пользователей похож на текущего пользователя
*  сортируем список

Мы получили ранжированный список фильмов, а также прогноз оценки, которую поставит Toby этим фильмам.

## Коллаборативная фильтрация по сходству объектов (Item-based collaborative filtering)

Мы реализовали механизм выдачи рекомендаций таким образом, что для создания набора данных необходимы оценки, выставленные каждым
пользователем. Для нескольких тысяч людей это, возможно, и будет работать, но, когда речь идет о миллионах пользователей и товаров, сравнение каждого пользователя со всеми другими, а затем сравнение товаров, займет недопустимо много времени.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1Vd3ofIheUgmxDKLq72cVPTPyl5IyAxo0"
alt="IMAGE ALT TEXT HERE" width="500" border="0" /></a>


Техника, которую мы применяли до сих пор, называется **коллаборативной фильтрацией по схожести пользователей**. Альтернатива известна под названием **коллаборативная фильтрация по схожести образцов**.

Основная идея процедуры фильтрации по схожести образцов заключается в том, чтобы для каждого образца заранее вычислить большинство похожих на него. Тогда для выдачи рекомендаций алгоритму достаточно будет найти те образцы, которым пользователь выставил наивысшие оценки, а затем сформировать список образцов, максимально похожих на эти.

Отметим одно существенное отличие: хотя на первом шаге необходимо исследовать все данные, результаты сравнения образцов изменяются не так часто, как результаты сравнения пользователей.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1VAoSpaUcbabIFRWlXqrBFDs7LhIISkA5"
alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>

Чтобы сравнивать образцы, нужно первым делом построить полный набор данных о похожих образцах. В первую очередь, нам нужно переделать словарь предпочтений критиков таким образом, чтобы теперь был словарь фильмов и их оценок:
```python
film_dict = {
    'film_name_1':
    {'critic_1': 'score_1',
     'critic_2': 'score_2',
     ...
    },
    ...
}
```
Далее для каждого фильма посчитать наиболее похожие образцы и коэффициенты похожести для них. Для подсчета схожести будем использовать евклидово расстояние.

Напишем функцию **calculate_similar_items()**, которая делает сразу оба шага. На вход её нужно подать словарь с оценками критиков *critics* и число образцов, которые мы считаем наиболее похожими (остальные не выводим).

In [ ]:
# Создать словарь, содержащий для каждого образца те образцы, которые больше всего похожи на него.
def calculate_similar_items(prefs, n=10):

    result = {}
    # обращаем матрицу предпочтений, чтобы строки соответствовали образцам
    itemPrefs = transformPrefs(prefs)

    c = 0
    for item in itemPrefs:
        # проверка состояния для больших наборов данных
        c = c + 1
        if c % 100 == 0: print("%d / %d" % (c,len(itemPrefs)))

        # найти образцы, максимально похожие на данный
        scores = top_matches(itemPrefs, item, n=n, similarity=sim_distance)
        result[item] = scores
    return result

In [ ]:
itemsim=calculate_similar_items(critics,n=10)
calculate_similar_items(critics,n=10)


{'Lady in the Water': [(0.7272727272727273, 'You, Me and Dupree'),
  (0.6666666666666666, 'The Night Listener'),
  (0.5882352941176471, 'Snakes on a Plane'),
  (0.46153846153846145, 'Just My Luck'),
  (0.3333333333333333, 'Superman Returns')],
 'Snakes on a Plane': [(0.5882352941176471, 'Lady in the Water'),
  (0.5833333333333333, 'Superman Returns'),
  (0.5714285714285714, 'The Night Listener'),
  (0.32, 'Just My Luck'),
  (0.24489795918367344, 'You, Me and Dupree')],
 'Just My Luck': [(0.47058823529411764, 'You, Me and Dupree'),
  (0.46153846153846145, 'Lady in the Water'),
  (0.42105263157894735, 'The Night Listener'),
  (0.32, 'Snakes on a Plane'),
  (0.21621621621621623, 'Superman Returns')],
 'Superman Returns': [(0.5833333333333333, 'Snakes on a Plane'),
  (0.4067796610169492, 'The Night Listener'),
  (0.3333333333333333, 'Lady in the Water'),
  (0.25263157894736843, 'You, Me and Dupree'),
  (0.21621621621621623, 'Just My Luck')],
 'You, Me and Dupree': [(0.7272727272727273, 'La

In [ ]:
pp.pprint(itemsim)

{   'Just My Luck': [   (0.2222222222222222, 'Lady in the Water'),
                        (0.18181818181818182, 'You, Me and Dupree'),
                        (0.15384615384615385, 'The Night Listener'),
                        (0.10526315789473684, 'Snakes on a Plane'),
                        (0.06451612903225806, 'Superman Returns')],
    'Lady in the Water': [   (0.4, 'You, Me and Dupree'),
                             (0.2857142857142857, 'The Night Listener'),
                             (0.2222222222222222, 'Snakes on a Plane'),
                             (0.2222222222222222, 'Just My Luck'),
                             (0.09090909090909091, 'Superman Returns')],
    'Snakes on a Plane': [   (0.2222222222222222, 'Lady in the Water'),
                             (0.18181818181818182, 'The Night Listener'),
                             (0.16666666666666666, 'Superman Returns'),
                             (0.10526315789473684, 'Just My Luck'),
                             (

Получили вложенный словарь с фильмами и их похожестью на другие фильмы.

Теперь мы готовы выдавать рекомендации, пользуясь словарем данных о схожести образцов. По аналогии с тем, что мы уже делали, необходимо получить список всех образцов, которым пользователь выставлял оценки, найти похожие и взвесить их с учетом коэффициентов подобия (умножить оценку на коэффициент подобия).

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1QIJpqC6zQkiQ-1-AzNSmvK4bc8fohsH1"
alt="IMAGE ALT TEXT HERE" width="500" border="0" /></a>

Реализуем функцию **get_recommended_items()**. На вход подается:
* словарь с оценками пользователя (в нашем случае это словарь критиков, *critics*)
* словарь с данными о схожести образцов, *itemsim*
* имя критика, для которого строим рекомендации

In [ ]:
def get_recommended_items(prefs, itemMatch, user):

    # инициализируем словари
    userRatings = prefs[user]
    scores = {}
    totalSim = {}

    # цикл по образцам, оцененным данным пользователем (хранятся в userRatings)
    for (item, rating) in userRatings.items():

        # цикл по образцам, похожим на данный (хранятся в словаре itemMatch)
        for (similarity, item2) in itemMatch[item]:
            # пропускаем объект, если пользователь уже оценивал данный образец
            if item2 in userRatings:
                continue
            # взвешенная суммы оценок, умноженных на коэффициент подобия
            scores.setdefault(item2, 0)
            scores[item2] = scores[item2] + similarity * rating
            # сумма всех коэффициентов подобия
            totalSim.setdefault(item2, 0)
            totalSim[item2] = totalSim[item2] + similarity
            if totalSim[item2] == 0:
                totalSim[item2] = 0.0000001  # чтобы избежать деления на ноль
    # делим каждую итоговую оценку на взвешенную сумму, чтобы вычислить среднее
    rankings = [(score / totalSim[item], item) for item, score in scores.items()]

    # возвращает список rankings, отсортированный по убыванию
    rankings.sort(reverse=True)
    return rankings

Получим рекомендации для пользователя Toby:

In [ ]:
get_recommended_items(critics,itemsim,'Toby')

[(3.2311953807740323, 'The Night Listener'),
 (2.855135135135135, 'Lady in the Water'),
 (2.7566953006568977, 'Just My Luck')]

## Метрики качества для рекомендательных систем

Для оценки качества работы рекомендательных систем можно рассмотреть две специальные метрики **Hit Rate** (HR) и **Mean Reciprocal Rank** (MRR).

Определим как **попадание** (**hit**) - ситуацию, когда фильм был рекомендован пользователю в топ-N и пользователь посмотрел его.

**Hit Rate (HR)**

**Hit Rate (процент попаданий)** определяется как общее число попаданий, нормированное на количество пользователей.

$$
\text{HR} = \frac{1}{\text{# test users}} \sum_{\text{test users}}{\text{hit}}, \quad
$$

Чем больше процент попаданий, тем лучше будет наша система рекомендаций.

*Замечание*: в некотором смысле, Hit Rate представляет собой производное от классических ML метрик precision и recall.

**Mean Reciprocal Rank (MRR)**

Отличие метрики **Mean Reciprocal Rank** от предыдущей в том, что мы учитываем не общее количество попаданий, а обратное рангу (месту в списке рекомендаций) **первое попадание**. Например, если фильм, просмотренный пользователем (попадание), стоял на втором месте, обратное рангу будет равно $\frac{1}{2}$, а если бы стоял на третьем - $\frac{1}{3}$ и так далее.

$$
\text{MRR} = \frac{1}{\text{# test users}} \sum_{\text{test users}}{\frac{1}{\text{hit rank}}}
$$

Рассмотрим на примере. Toby получил следующий список рекомендаций:

**Code 18:**

In [ ]:
get_recommendations(critics,'Toby', sim_pearson)

[(3.347789526713101, 'The Night Listener'),
 (2.832549918264162, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

Допустим, после этого Toby посмотрел 5 фильмов и выставил им следующие оценки:


|Score|Film|
|----|--------------------|
|4.5 | The Departed |
|4   | The Night Listener |
|3.9 | Lady in the Water |
|3.7 | The Firm |
|3   | Just My Luck |

Как мы видим, только два фильма из списка рекомендаций попали в личный топ-3 Toby. Получается, что **попадание (hits)** у него 2.

**HR = $\dfrac{1}{1} * 2 = 2$**, так как выборка тестовых пользователей (test\_users) состоит из одного Toby.  

* **$\frac{1}{hit\_rank}$** для фильма The Night Listener - это $\dfrac{1}{2}$
* **$\frac{1}{hit\_rank}$** для фильма Lady in the Water - это $\dfrac{1}{3}$

Но фильм **The Night Listener** стоит выше в рейтинге Toby, чем **Lady in the Water**, поэтому:

**MRR = $\dfrac{1}{1}*\dfrac{1}{2} = \dfrac{1}{2}$**, так как, опять же, выборка тестовых пользователей (test\_users) состоит из одного Toby.  

## Бонус* : Рекомендация на данных MovieLens

В этом примере мы рассмотрим реальный набор данных с оценками фильмов, который называется MovieLens. Источник данных: http://grouplens.org/datasets/movielens/

Датасет представляет собой набор фильмов, которые 162541 пользователей сайта MovieLens оценивали по 5-бальной шкале. В наборе также есть дополнительная информация такая как описание фильмов тегами, но в нашем упражнении мы её использовать не будем для упрощения. Также, чтобы уменьшить время вычислений, будем использовать данный только 943 пользователей. Каждый пользователь в датасете оценил минимум 20 фильмов.

Загруженный датасет будет в следующем виде:

```python

preferences = {
    'user_id_1' : {
        'film_name_1' : rate,
        'film_name_2' : rate,
        ...
        'film_name_21' : rate,
    },
    ...
    'user_id_943' : {
        'film_name_1' : rate,
        'film_name_2' : rate,
        ...
        'film_name_30' : rate,
    }
}

```


Чтобы загрузить набор данных MovieLens, в модуле **recommendations** есть функция **loadMovieLens()**. Она возвращает словарь предпочтений фильмов для набора людей. Пользователи зашифрованы идентификатором. Для каждого пользователя имеется название
фильма, а также оценка, выставленная фильму данным пользователе.

**Code 19:**

In [ ]:
%%capture
!wget https://www.dropbox.com/sh/ob1qyfzqygkc6ph/AADg2G0By0EuRZCLR5gSAmP7a

In [ ]:
%%capture
!unzip AADg2G0By0EuRZCLR5gSAmP7a -d data

In [ ]:
# Функция для загрузки данных MovieLens
def loadMovieLens(path='data/'):
    # получить названия фильмов
    movies = {}
    for line in open(path+'/u.item', encoding = "ISO-8859-1"):
        (id, title) = line.split('|')[0:2]
        movies[id] = title
    # загрузить данные
    prefs = {}
    for line in open(path+'/u.data'):
        (user, movieid, rating, ts) = line.split('\t')
        prefs.setdefault(user, {})
        prefs[user][movies[movieid]] = float(rating)
    return prefs

In [ ]:
preferences = loadMovieLens()

Выведем фильмы для пользователя с ID 87 и рейтинги этих фильмов

**Code 20:**

In [ ]:
pp.pprint(preferences['87'])

{   '2001: A Space Odyssey (1968)': 5.0,
    'Ace Ventura: Pet Detective (1994)': 4.0,
    'Addams Family Values (1993)': 2.0,
    'Addicted to Love (1997)': 4.0,
    'Adventures of Priscilla, Queen of the Desert, The (1994)': 3.0,
    'Adventures of Robin Hood, The (1938)': 5.0,
    'Air Force One (1997)': 3.0,
    'Air Up There, The (1994)': 3.0,
    'Alien (1979)': 4.0,
    'American President, The (1995)': 5.0,
    'Annie Hall (1977)': 4.0,
    'Apocalypse Now (1979)': 4.0,
    'Babe (1995)': 5.0,
    'Baby-Sitters Club, The (1995)': 2.0,
    'Back to the Future (1985)': 5.0,
    'Bad Boys (1995)': 4.0,
    'Bananas (1971)': 5.0,
    'Barcelona (1994)': 3.0,
    'Batman & Robin (1997)': 4.0,
    'Batman (1989)': 3.0,
    'Batman Returns (1992)': 3.0,
    'Big Green, The (1995)': 3.0,
    'Big Squeeze, The (1996)': 2.0,
    'Birdcage, The (1996)': 4.0,
    'Blade Runner (1982)': 4.0,
    'Blues Brothers, The (1980)': 5.0,
    'Boomerang (1992)': 3.0,
    'Boot, Das (1981)': 4.0,
   

Посмотрим, какие рекомендации получим с помощью коллаборативной фильтрации по схожести пользователей. Напомним, что для этого реализована функция **getRecommendations()**, которая выдает рекомендации для данного пользователя по похожести на других и предположительную его оценку фильму.

Функция принимает на вход следующие аргументы:
* словарь пользователей, в этом случае это *preferences*
* имя персоны *person*, для которой подбираем соответствия
* имя функции-критерия для сравнения

Выведем первые топ-30 рекомендаций для пользователя с ID 87, для этого в квадратных скобочках укажем [0:30].

*Замечание*: может появиться предупреждение RuntimeWarning, это связано с тем, что теперь мы работаем с большой выборкой и при подсчете корреляции Пирсона может возникать деление на ноль. В этом случае мы присваиваем корреляции значение ноль.

**Code 21:**

In [ ]:
get_recommendations(preferences, '87', sim_pearson)[0:30]

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Boys, Les (1997)'),
 (4.898844431289231, 'Legal Deceit (1997)'),
 (4.815019082242706, 'Letter From Death Row, A (1998)'),
 (4.732108298394142, 'Hearts and Minds (1996)'),
 (4.696244466490867, 'Pather Panchali (1955)'),
 (4.652397061026759, 'Lamerica (1994)'),
 (4.538723693474813, 'Leading Man, The (1996)'),
 (4.535081339106105, 'Mrs. Dalloway (1997)'),
 (4.532337612572983, 'Innocents, The (1961)'),
 (4.527998574747078, 'Casablanca (1942)'),
 (4.510270149719863, 'Everest (1998)'),
 (4.49396775542844, 'Dangerous Beauty (1998)'),
 (4.485151301801341, 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.463287461290223, 'Wrong Trousers, The (1993)'),
 (4.4509794369410

Интересно также сравнить полученный результат с рекомендациями по схожести образцов. Напомним, чтобы сравнить образцы, нужно в первую очередь перестроить наш набор данных из набора предпочтений пользователей в набор оценок фильмов. Для этого используем функцию **calculateSimilarItems()**. На вход её нужно подать словарь с оценками пользователей *prefernces*. Кроме этого, мы передадим аргумент *n = 50*, так как хотим оставить только 50 наиболее похожих образцов в новом словаре.  

В этот раз мы работаем с большим набором данных и вычисления могут занять некоторое время, поэтому функция также будет выводить сообщение о том, сколько фильмов она уже просмотрела.

**Code 22:**

In [ ]:
item_similarities = calculate_similar_items(preferences, n=50)

100 / 1664
200 / 1664
300 / 1664
400 / 1664
500 / 1664
600 / 1664
700 / 1664
800 / 1664
900 / 1664
1000 / 1664
1100 / 1664
1200 / 1664
1300 / 1664
1400 / 1664
1500 / 1664
1600 / 1664


Посмотрим на результат:

**Code 23:**

In [ ]:
item_similarities["What's Eating Gilbert Grape (1993)"]

[(1.0, 'Yankee Zulu (1994)'),
 (1.0, 'Woman in Question, The (1950)'),
 (1.0, 'Witness (1985)'),
 (1.0, "Wend Kuuni (God's Gift) (1982)"),
 (1.0, 'Vie est belle, La (Life is Rosey) (1987)'),
 (1.0, 'Vermont Is For Lovers (1992)'),
 (1.0, 'Van, The (1996)'),
 (1.0, 'Two or Three Things I Know About Her (1966)'),
 (1.0, 'Two Much (1996)'),
 (1.0, 'Two Deaths (1995)'),
 (1.0, 'Two Bits (1995)'),
 (1.0, 'Touki Bouki (Journey of the Hyena) (1973)'),
 (1.0, 'Total Eclipse (1995)'),
 (1.0, 'To Have, or Not (1995)'),
 (1.0, 'To Cross the Rubicon (1991)'),
 (1.0, 'Tigrero: A Film That Was Never Made (1994)'),
 (1.0, 'The Courtyard (1995)'),
 (1.0, 'Terror in a Texas Town (1958)'),
 (1.0, 'Talking About Sex (1994)'),
 (1.0, 'T-Men (1947)'),
 (1.0, 'Symphonie pastorale, La (1946)'),
 (1.0, 'Swept from the Sea (1997)'),
 (1.0, 'Stranger in the House (1997)'),
 (1.0, 'Stonewall (1995)'),
 (1.0, 'Squeeze (1996)'),
 (1.0, 'Spirits of the Dead (Tre passi nel delirio) (1968)'),
 (1.0, 'Somebody to Love

Теперь получим топ-30 рекомендаций для пользователя с учетом похожести фильмов. Используем функцию **getRecommendedItems()**, которой на вход подается:
* словарь с оценками пользователя (*preferences*)
* словарь с данными о схожести фильмов, *item_similarities*
* ID пользователя, для которого строим рекомендации

**Code 24:**

In [ ]:
getRecommendedItems(preferences, item_similarities, '87')[0:30]

[(5.0, "What's Eating Gilbert Grape (1993)"),
 (5.0, 'Vertigo (1958)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Toy Story (1995)'),
 (5.0, 'Titanic (1997)'),
 (5.0, 'Sword in the Stone, The (1963)'),
 (5.0, 'Stand by Me (1986)'),
 (5.0, 'Sling Blade (1996)'),
 (5.0, 'Silence of the Lambs, The (1991)'),
 (5.0, 'Shining, The (1980)'),
 (5.0, 'Shine (1996)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, 'Scream (1996)'),
 (5.0, 'Rumble in the Bronx (1995)'),
 (5.0, 'Rock, The (1996)'),
 (5.0, 'Robin Hood: Prince of Thieves (1991)'),
 (5.0, 'Rising Sun (1993)'),
 (5.0, 'Reservoir Dogs (1992)'),
 (5.0, 'Replacement Killers, The (1998)'),
 (5.0, 'Police Story 4: Project S (Chao ji ji hua) (1993)'),
 (5.0, 'Murder in the First (1995)'),
 (5.0, 'Houseguest (1994)'),
 (5.0, 'Great Dictator, The (1940)'),
 (5.0, 'Friday (1995)'),
 (5.0, 'Day the Sun Turned Cold, The (Tianguo niezi) (1994)'),
 (5.0, 'Chamber, The (1996)'),
 (5.0, 'Body Snatcher, The (1945)'),
 (5.0, 'Big Lebowski, The (19

Выведем список фильмов, которые пользователь с id `87` оценил максимальным баллом:

**Code 25:**

In [ ]:
{x: preferences['87'][x] for x in preferences['87'] if preferences['87'][x] == 5}

{'Jurassic Park (1993)': 5.0,
 'Indiana Jones and the Last Crusade (1989)': 5.0,
 'Good, The Bad and The Ugly, The (1966)': 5.0,
 'Dead Poets Society (1989)': 5.0,
 'M*A*S*H (1970)': 5.0,
 '2001: A Space Odyssey (1968)': 5.0,
 'Four Weddings and a Funeral (1994)': 5.0,
 'Prefontaine (1997)': 5.0,
 'This Is Spinal Tap (1984)': 5.0,
 'Cool Hand Luke (1967)': 5.0,
 'True Lies (1994)': 5.0,
 'Sleepless in Seattle (1993)': 5.0,
 'Young Frankenstein (1974)': 5.0,
 'Fugitive, The (1993)': 5.0,
 'In the Line of Fire (1993)': 5.0,
 'Speed (1994)': 5.0,
 'Terminator 2: Judgment Day (1991)': 5.0,
 'Shawshank Redemption, The (1994)': 5.0,
 'Clear and Present Danger (1994)': 5.0,
 'Dances with Wolves (1990)': 5.0,
 'Adventures of Robin Hood, The (1938)': 5.0,
 'Blues Brothers, The (1980)': 5.0,
 'Babe (1995)': 5.0,
 'Top Gun (1986)': 5.0,
 'Get Shorty (1995)': 5.0,
 'Wizard of Oz, The (1939)': 5.0,
 'When Harry Met Sally... (1989)': 5.0,
 'Star Trek IV: The Voyage Home (1986)': 5.0,
 'Net, The (199

Валидация для моделей рекомендательных систем устроена достаточно сложно.
Например, нам нужно забыть часть рейтингов фильмов для пользователя, а потом попытаться восстановить их с
помощью нашей модели.

В данном случае, мы можем оценить адекватность полученных рекомендаций, просмотрев списки фильмов. Однако, на текущем датасете, из-за размера выборки, такое сделать сложно.

Приведем примеры интерпретации:
* В случае использования расстояния, основанного на коррелции, модель рекомендует в первую очередь боевики
* В случае использование похожестей фильмов мы видим как общепризнанные шедевры, так и боевики
* В реальности пользователю нравятся фильмы, которые нравятся всем, приключенческие фильмы и боевики

С такой точки зрения оба списка рекомендаций кажутся адекватным, а то, что они не пересекаются связано с тем,
что есть много похожих друг на друга фильмов - и мы не можем рекомендовать сразу все такие фильмы пользователю.


Для рекомендательных систем есть специальные библиотеки, например, https://surprise.readthedocs.io/, однако их применение требует более глубокого знакомства с темой.